In [ ]:
import anndata as ad
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import matplotlib.colors as mcolors
import scipy.sparse as sp
import pingouin as pg

In [ ]:
counts_data = pd.read_csv("counts_data.csv", index_col=0)
counts_data.columns = counts_data.columns.str.replace('^SeuratProject', '', regex=True)
counts_data = counts_data.T
metadata = pd.read_csv("metadata.csv", index_col=0)
metadata.index = metadata.index.str.replace('^SeuratProject', '', regex=True)

adata = ad.AnnData(X=counts_data.values, obs=metadata, var=pd.DataFrame(index=counts_data.columns))
adata = adata[adata.obs['sample'] == 222107]

types = pd.read_csv('adpkd_clustering.csv')
types['Barcode'] = types['Barcode'] + '_2'
types.index = types['Barcode']
types = types.drop('Barcode', axis = 1)
adata.obs['cell_types'] = types['Kidney group 222107 analysis']

adata = adata[adata.obs['cell_types'].isna() == False]

In [ ]:
def gene_correlation_pingouin(adata, genes):
    genes = [g for g in genes if g in adata.var_names]
    data = adata[:, genes].X
    if not isinstance(data, np.ndarray):
        data = data.toarray()

    df = pd.DataFrame(data, columns=genes)


    corr_res = pg.pairwise_corr(df, method='pearson', padjust='fdr_bh')


    corr_matrix = df.corr(method='pearson')
    pval_matrix = pd.DataFrame(np.ones((len(genes), len(genes))), index=genes, columns=genes)
    for _, row in corr_res.iterrows():
        pval_matrix.loc[row['X'], row['Y']] = row['p-corr']
        pval_matrix.loc[row['Y'], row['X']] = row['p-corr']

    np.fill_diagonal(pval_matrix.values, 0)

    return corr_matrix, pval_matrix

In [ ]:
coi = adata[adata.obs['cell_types'] == 'COI']

In [ ]:
selected_genes = ['PCNA', 'MCM27', 'PLK1', 'MKI67','FEN1','RRM1','RRM2','CDK1','RPA2','RFC4','RFC2','PRIM2',
                     'POLA1','RFC3','RPA1','RPA3','RFC5', 'SMC3', 'STAG2', 'SMC1A', "KRT7", 'KRT17', "SLPI", "TACSTD2", "ITGB6", "MMP7", "COL1A1",
    "KRT19", "CLDN4", "NNMT" ]

corr, pvals = gene_correlation_pingouin(coi, selected_genes)

In [ ]:
filtered = pvals[(pvals < 0.05) & (pvals > 0)]
pvals  = filtered.loc[["KRT7", 'KRT17', "SLPI", "TACSTD2", "ITGB6", "MMP7", "COL1A1", "KRT19", "CLDN4", "NNMT" ]][['PCNA', 'MKI67','RRM1','CDK1','POLA1','RPA1','SMC3', 'STAG2', 'SMC1A']]
corr = corr.loc[["KRT7", 'KRT17', "SLPI", "TACSTD2", "ITGB6", "MMP7", "COL1A1", "KRT19", "CLDN4", "NNMT" ]][['PCNA', 'MKI67','RRM1','CDK1','POLA1','RPA1','SMC3', 'STAG2', 'SMC1A']]

In [ ]:
p = pvals.loc[corr.index, corr.columns]

annot = corr.copy()
for i in corr.index:
    for j in corr.columns:
        r = corr.loc[i, j]
        pval = p.loc[i, j]
        if pd.isna(r) or pd.isna(pval):
            annot.loc[i, j] = ""
        else:
            annot.loc[i, j] = f"{r:.2f}\np={pval:.2g}"

In [ ]:
plt.figure(figsize=(9, 3))
sns.heatmap(
    corr,
    annot=annot,
    fmt="",
    cmap='viridis',
    vmin=0,
    vmax=0.3,
    cbar=True,
    mask = pvals.isna()
)
plt.title('Correlation Heatmap with Corresponding p-values (ADPKD Data)', fontsize=12, fontweight='bold')

plt.xticks(fontsize=10, fontweight='bold')
plt.yticks(fontsize=10, fontweight='bold', rotation=0)

plt.tight_layout()
plt.show()